In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv
from matplotlib import pyplot as plt

from utils.s3_bucket import S3Bucket
from utils.predictors import ImageClassificationPredictor

In [ ]:
load_dotenv("env")

In [ ]:
model_predictor = ImageClassificationPredictor(
    model_id=os.environ["IMAGE_CLASSIFICATION_MODEL_ID"],
    model_version=os.environ["IMAGE_CLASSIFICATION_MODEL_VERSION"],
    model_uri=os.environ["IMAGE_CLASSIFICATION_MODEL_URI"],
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"]
)

In [ ]:
%%time
model_predictor.deploy(
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
    instance_count=int(os.environ["INFERENCE_INSTANCE_COUNT"]),
)

In [ ]:
bucket_name = "ava-cv-raw-photo-bucket"
prefix = "test"

bucket = S3Bucket(
    bucket_name=bucket_name,
    region_name=os.environ["REGION_NAME"]
)
for obj in bucket.filter(prefix):
    key = obj.key
    image = bucket[key]
    labels, probabilities = model_predictor.predict(image)
    plt.figure(figsize=(5, 5))
    plt.imshow(np.array(image))
    plt.show()
    print(os.path.join("s3://", bucket_name, key))
    for label, probability in zip(labels, probabilities):
        print(f"{label}: {probability*100:.2f}%")
    print("")

In [ ]:
# Delete the SageMaker endpoint and the attached resources
model_predictor.delete()